## 💡 Coreset (CRAIG) 子集選擇演算法筆記

CRAIG (Coreset by RelAtIve Gradients) 方法是一種資料效率學習技術，用於從大型資料集中高效地選取一個具備代表性的子集（Coreset）。

### 🎯 演算法核心目標

* 找到一個包含 $r$ 個樣本的子集 $\mathcal{S} \subset \mathcal{D}$，以及一組權重 $\mathbf{\gamma}$。
* 目標是使在 **加權子集 $\mathcal{S}$** 上訓練的模型行為（如累積梯度、最終性能）與在 **完整資料集 $\mathcal{D}$** 上訓練的結果**儘可能接近**。

> 📌 **核心關注點：** Coreset 選擇基於保留**梯度資訊**。每個樣本 $i$ 被表示為一個與梯度相關的向量 $\mathbf{g}_i$。

### 📊 核心特徵向量 $\mathbf{g}$

* **定義：** $\mathbf{g}_i$ 是模型在預熱階段（或隨機初始化）計算的 **最後一層的梯度近似向量**。
* **計算方式 (Cross-Entropy 損失近似)：**
    $$\mathbf{g} = \mathbf{p} - \mathbf{e}_y$$
    其中：
    * $\mathbf{p}$：模型的 Softmax 輸出機率。
    * $\mathbf{e}_y$：目標類別 $y$ 的 One-hot 向量。

### 🛠️ 演算法步驟 (Facility Location 貪婪選擇)

Coreset 的選擇採用基於 Facility Location 目標的**貪婪**策略：

| 步驟 | 描述 | 目的 |
| :--- | :--- | :--- |
| **1. 特徵向量化 ($\mathbf{g}$)** | 計算所有樣本的 $\mathbf{g}_i$ 向量（例如：`get_last_layer_grads_like` 函式）。 | 將樣本轉換為梯度空間中的點。 |
| **2. 子集大小分配** | 將總 Coreset 大小 $r_{total}$ 根據**類別比例**分配給每個類別 $c$，得到 $r_c$。 | 解決類別不平衡，確保資料代表性。 |
| **3. Facility Location 貪婪選擇** | 在 $\mathbf{g}$ 空間中，從每個類別 $c$ 內**貪婪**選取 $r_c$ 個代表點 $\mathcal{S}_c$。**目標是最小化所有非選定點到最近選定點的距離總和**。 | 確保選定的 Coreset 點能最大化地**覆蓋**整個資料空間的分佈（梯度空間）。 |
| **4. 權重分配 $\mathbf{\gamma}$** | 將完整資料集中的**每個原始點**分配給 Coreset 中**最近的代表點 $j$**。代表點 $j$ 的權重 $\gamma_j$ 即為分配給它的原始點數量。 | 賦予 Coreset 點正確的權重，以保留原始資料集的分佈信息。 |

### ✨ 訓練階段的應用

* **資料載入器：** 使用 `WeightedRandomSampler`。
* **權重：** 將 $\mathbf{\gamma}$ 向量作為採樣權重。
* **效果：** 權重 $\mathbf{\gamma}$ 確保了在訓練時，影響力大（代表樣本多）的 Coreset 點會被**更頻繁地採樣**，從而在數據量大幅減少的情況下，依然能近似完整資料集的訓練路徑。

### 🖥️ 底下實戰區

Set up

In [5]:
!pip -q install torch torchvision numpy tqdm

import torch, torchvision, numpy as np, time, math
from torch import nn
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torchvision import datasets, transforms
from tqdm import tqdm
torch.manual_seed(0); np.random.seed(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'


Load Data


In [6]:
transform = transforms.Compose([transforms.ToTensor()])
train_full = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
idx_all = np.random.permutation(len(train_full))[:10000]
train_10k = Subset(train_full, idx_all)
test_ds = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

def make_loader(ds, batch_size=128, shuffle=True, sampler=None):
    return DataLoader(ds, batch_size=batch_size, shuffle=(shuffle and sampler is None), sampler=sampler)


 Define Model

In [7]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()       # 將輸入的2D圖像攤平為1D向量
        self.fc1 = nn.Linear(28*28, 100)    # 第一層全連接層
        self.act = nn.Sigmoid()         # Sigmoid激活函數
        self.fc2 = nn.Linear(100, 10)      # 第二層全連接層，logits
    def forward(self, x):
        x = self.flatten(x)
        zL = self.fc2(self.act(self.fc1(x)))   # 通過第二個線性層生成10維輸出
        return zL                 # logits before softmax，返回的zL是Logits

def accuracy(logits, y):
    # 找出每個樣本分數最高的索引，將預測類別與真實標籤y進行元素級比較。
    return (logits.argmax(dim=1) == y).float().mean().item()


Compute last-layer gradient-like vectors (p - y)

In [8]:
@torch.no_grad()
# 計算每個樣本的梯度近似向量 g=p-e_y，並將所有批次的結果堆疊 (concatenate) 成一個大的 Tenso
def get_last_layer_grads_like(model, loader):
    model.eval()
    G, Y, Xidx = [], [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        logits = model(xb)
        p = logits.softmax(dim=1)
        # ey 是目標 one-hot 向量 (B, 10)
        ey = torch.zeros_like(p).scatter_(1, yb.view(-1,1), 1.0)
        # g 是梯度近似向量 (p - ey) (B, 10)
        # 關鍵輸入，g代表了每個樣本的「資訊價值」或「影響力」
        g = (p - ey).cpu()              # (B, 10)
        G.append(g); Y.append(yb.cpu());
    # ================= TODO =======================
    G = torch.cat(G, dim=0).numpy() # 將所有批次的 (B, 10) 向量堆疊成 (N, 10)，並轉為 NumPy 數組
    Y = torch.cat(Y, dim=0).numpy() # 將所有批次的標籤堆疊成 (N,)，並轉為 NumPy 數組
    # ================= END =======================
    return G, Y

# Initialize a random model and run forward once to obtain g vectors (no backprop needed)
warm_model = MLP().to(device)
warm_loader = make_loader(train_10k, batch_size=256, shuffle=False)
G, Y = get_last_layer_grads_like(warm_model, warm_loader)  # G: (N,10)

Facility-location greedy selection

In [9]:
# 目標是找到一組最具代表性的點 S，以便它們能很好地“覆蓋”整個資料集在梯度空間中的分佈。

# 獨立執行貪婪選擇
# 找到r個Coreset點，使得所有原始點到最近設施的總距離最小。
def greedy_facloc_indices(Gc, r):
    # Gc: (Nc, d)
    Nc = Gc.shape[0]
    S = []
    best = np.full(Nc, -np.inf, dtype=np.float32)  # current best similarity per point
    # Avoid full O(N^2) distance matrix; compute in small batches to save RAM
    # 相似度函式
    def sim_to_point(j):
        # s(i,j) = -||g_i - g_j||
        diff = Gc - Gc[j:j+1]
        return -np.sqrt((diff*diff).sum(axis=1))
    # Initialize: choose one center (similar to k-means++: pick closest to mean)
    mu = Gc.mean(axis=0)
    # 選擇最接近整個類別平均值mu的點，作為初始中心點j_0。
    j0 = np.argmin(((Gc - mu)**2).sum(axis=1))
    s = sim_to_point(j0)
    best = np.maximum(best, s)
    S.append(j0)
    # 在每一步中，該演算法必須從所有未被選中的候選點j中，選出能提供最大邊際增益的點。
    while len(S) < r:
        # For each candidate j, compute marginal gain sum(max(best, s_·j) - best)
        # Use sampling approximation to accelerate: randomly sample candidates,
        # then compute full gain only for top-k
        cand = np.setdiff1d(np.arange(Nc), np.array(S), assume_unique=True)
        probe = np.random.choice(cand, size=min(2000, cand.size), replace=False)
        est_gain = []
        for j in probe:
            s = sim_to_point(j)
            est_gain.append(np.sum(np.maximum(best[probe], s[probe]) - best[probe]))
        # 計算完整增益 (Full Gain Calculation)
        topk_idx = np.argsort(est_gain)[-10:]
        topk = probe[topk_idx]
        # Compute full gain among top-k
        gains = []
        for j in topk:
            s = sim_to_point(j)
            gains.append(np.sum(np.maximum(best, s) - best))
        # 選出最佳點argmax
        j_star = topk[np.argmax(gains)]
        s = sim_to_point(j_star)
        # 更新每個點到新集合的最高相似度
        best = np.maximum(best, s)
        # 將新選出的 Coreset 點加入集合
        S.append(j_star)
    return np.array(S, dtype=int)

# 高層流程與權重分配任務
def classwise_coreset(G, Y, r_total=2000):
    # Allocate r_total based on the class proportions of the original dataset
    # 按比例分配 Coreset 大小，確保 Coreset 能夠處理類別不平衡問題。
    # 找出每個類別的索引
    idx_by_c = [np.where(Y==c)[0] for c in range(10)]
    # 各類別樣本數
    Ns = np.array([len(idxs) for idxs in idx_by_c])
    # 根據比例計算每個類別 r_c 的大小
    share = np.maximum(1, np.round(r_total * Ns / Ns.sum()).astype(int))
    S_all, assign_gamma = [], np.zeros(len(Y), dtype=int)
    for c in range(10):
        Gc = G[idx_by_c[c]]
        Sc_local = greedy_facloc_indices(Gc, r=share[c])
        Sc_global = idx_by_c[c][Sc_local]
        S_all.append(Sc_global)
        # Compute γ (number of points assigned to each representative)
        rep = G[Sc_global]
        # Assign each original point to its nearest selected representative
        # Batch processing to avoid O(Nr) memory blow-up
        for chunk in np.array_split(idx_by_c[c], max(1, len(idx_by_c[c])//2000)):
            # 計算所有原始點到所有代表點之間的歐氏距離平方
            D = ((G[chunk, None, :] - rep[None, :, :])**2).sum(axis=2) # (m, r_c)
            # 找出每個原始點最近的代表點的索引
            nearest = D.argmin(axis=1)
            for j in nearest:
                # 遞增該代表點的計數器（權重γ）
                assign_gamma[Sc_global[j]] += 1
    S_all = np.concatenate(S_all)
    gamma = assign_gamma[S_all]
    return S_all, gamma

# Example: select 2,000 samples (~20%) as coreset
S_idx, gamma = classwise_coreset(G, Y, r_total=2000)
len(S_idx), gamma.sum()  # gamma sum should equal 10,000 (or number of total samples)


(2001, np.int64(10000))

Build data loaders

In [10]:
train_10k_loader = make_loader(train_10k, batch_size=128, shuffle=True)

# Coreset subset (insert γ into sampler weights, so that each representative
# is sampled proportionally to its assigned number of original samples)
coreset_ds = Subset(train_10k, S_idx)
weights = torch.tensor(gamma, dtype=torch.float32)
sampler = WeightedRandomSampler(weights=weights, num_samples=int(gamma.sum().item()), replacement=True)
coreset_loader = make_loader(coreset_ds, batch_size=128, shuffle=False, sampler=sampler)

test_loader = make_loader(test_ds, batch_size=512, shuffle=False)

Training and evaluation

In [11]:
def train_eval(model, loader, epochs=5, use_gamma_in_loss=False):
    model = model.to(device)
    opt = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)
    ce = nn.CrossEntropyLoss(reduction='none' if use_gamma_in_loss else 'mean')
    t0 = time.time()
    for ep in range(epochs):
        model.train()
        for xb, yb in loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            if use_gamma_in_loss and hasattr(loader, 'sampler') and isinstance(loader.sampler, WeightedRandomSampler):
                # The sampler does not directly provide γ;
                # we approximate by batch frequency (average number of times each representative appears)
                # Simplification: do not multiply by γ, since weighted sampling already reflects γ
                loss = ce(logits, yb).mean()
            else:
                loss = ce(logits, yb).mean()
            opt.zero_grad(); loss.backward(); opt.step()
    t1 = time.time()

    # Evaluation phase
    model.eval()
    accs, losses = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            losses.append(nn.CrossEntropyLoss()(logits, yb).item())
            accs.append(accuracy(logits, yb))
    return {'time_sec': t1 - t0, 'test_acc': np.mean(accs), 'test_loss': np.mean(losses), 'model': model}

# A) Full dataset (10k)
res_full = train_eval(MLP(), train_10k_loader, epochs=5)
# B) Coreset only (weighted sampling by γ), same number of epochs
res_core = train_eval(MLP(), coreset_loader, epochs=5)

res_full, res_core

({'time_sec': 5.443616151809692,
  'test_acc': np.float64(0.8296300560235977),
  'test_loss': np.float64(0.7284958481788635),
  'model': MLP(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=784, out_features=100, bias=True)
    (act): Sigmoid()
    (fc2): Linear(in_features=100, out_features=10, bias=True)
  )},
 {'time_sec': 5.876196384429932,
  'test_acc': np.float64(0.8365866273641587),
  'test_loss': np.float64(0.7168105572462082),
  'model': MLP(
    (flatten): Flatten(start_dim=1, end_dim=-1)
    (fc1): Linear(in_features=784, out_features=100, bias=True)
    (act): Sigmoid()
    (fc2): Linear(in_features=100, out_features=10, bias=True)
  )})

Print results

In [12]:
print(f"[Full 10k]  time={res_full['time_sec']:.1f}s  acc={res_full['test_acc']:.4f}  loss={res_full['test_loss']:.4f}")
print(f"[Coreset ]  time={res_core['time_sec']:.1f}s  acc={res_core['test_acc']:.4f}  loss={res_core['test_loss']:.4f}")


[Full 10k]  time=5.4s  acc=0.8296  loss=0.7285
[Coreset ]  time=5.9s  acc=0.8366  loss=0.7168


## 🔬 Coreset 選擇實驗結果分析

本實驗比較了在完整資料集 (10,000 筆樣本) 和加權 Coreset (2,000 筆樣本, 約 20%) 上訓練 MLP 模型 5 個 Epoch 的性能。

### 1. 訓練結果比較表

| 指標 | 全資料集 (10k) | Coreset (2k, 加權) | 差異 | 分析 |
| :--- | :--- | :--- | :--- | :--- |
| **Test Accuracy** | $0.8296$ | $\mathbf{0.8366}$ | $\color{green}{+0.0070}$ | **Coreset 略勝**：顯示 Coreset 不僅保留了代表性，可能還過濾了冗餘或低影響的數據，提升了模型的泛化能力。 |
| **Test Loss** | $0.7285$ | $\mathbf{0.7168}$ | $\color{green}{-0.0117}$ | **Coreset 略勝**：損失更低，進一步證實了 Coreset 訓練的模型在測試集上表現更好。 |
| **Time (訓練時間)** | $\mathbf{5.4s}$ | $5.9s$ | $\color{red}{+0.5s}$ | **全資料集勝**：時間略長是因 Coreset 採用 $\mathbf{\gamma}$ **加權隨機採樣**。總採樣次數（梯度更新次數）與全資料集相似 (約 10,000 次)，額外開銷來自採樣器的複雜性。 |

### 2. 結論：數據高效性的驗證

#### A. 訓練性能提升 (數據品質高於數量)

* **數據高效性成功 (Data-efficiency)**：僅使用 **20%** 的獨特樣本，卻達成了**更高**的準確度和**更低**的損失。
* **原理**：Coreset 演算法（基於 Facility Location 和 $\mathbf{g} = \mathbf{p} - \mathbf{e}_y$ 梯度近似）成功地從數據集中「精選」出了對模型優化最有影響力的關鍵樣本。

#### B. 訓練時間的權衡 (採樣模擬)

* Coreset 的優勢在於**減少需要儲存和處理的數據量 (2K vs 10K)** 和**提高數據品質**。
* 在本實驗中，由於我們設置了相同的 Epoch 數，且 Coreset Loader 使用 $\mathbf{\gamma}$ 權重**模擬**了完整資料集的總採樣次數，因此總訓練時間保持在同一量級。
